In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import pprint, pickle
from collections import Counter

In [111]:
input_file = open('acoustic_dict_byDecade.dat','rb')
data2 = pickle.load(input_file) 

In [113]:
data2['1950s']

,Genre,AVG_LOUDNESS,AVG_TEMPO,AVG_DURATION,AVG_TIMBRE1,AVG_TIMBRE2,AVG_TIMBRE3,AVG_TIMBRE4,AVG_TIMBRE5,AVG_TIMBRE6,...,AVG_VAR_TIMBRE3,AVG_VAR_TIMBRE4,AVG_VAR_TIMBRE5,AVG_VAR_TIMBRE6,AVG_VAR_TIMBRE7,AVG_VAR_TIMBRE8,AVG_VAR_TIMBRE9,AVG_VAR_TIMBRE10,AVG_VAR_TIMBRE11,AVG_VAR_TIMBRE12
0,alternative,-0.905487,6.032769,7.410714,1.890789,-4.308844,0.572642,-0.165074,-0.355252,0.009320,...,103.537575,72.578626,49.895455,41.109196,19.838475,27.626424,33.735235,17.353084,13.227102,9.715683
1,blues,-11.176968,110.382080,150.795649,41.514085,-9.025085,32.812912,5.013468,-2.641550,6.301973,...,2503.430405,2553.850999,1002.983122,1267.118190,731.263683,539.738146,565.195334,328.118093,424.722423,369.180263
2,country,-7.316743,41.261322,73.237516,15.157835,-30.598889,3.389671,-2.794526,-7.305554,-0.528303,...,1296.222633,1341.657337,651.559347,506.191458,376.419509,284.708321,429.153246,173.161227,183.396912,199.211572
3,dance,-9.304844,119.589062,134.554378,41.680341,5.695987,25.197664,0.117832,-1.250763,4.758743,...,1830.225462,1688.576675,767.607447,972.604468,631.731490,381.511271,469.342112,254.708264,328.986202,353.552098
4,disco,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,electronic,-4.556077,47.401385,55.638632,16.392204,-10.492532,8.195926,1.763573,-1.594704,1.771470,...,657.957887,561.961304,355.060766,333.043844,204.201473,165.372452,211.146809,99.905553,130.475001,102.880016
6,folk,-12.345360,90.811180,128.827223,31.259306,-10.003800,4.468284,-4.202770,2.532394,0.588434,...,2655.931780,2694.029642,1021.906377,1018.234048,747.630875,647.240437,631.903102,364.349140,380.602340,296.654951
7,indie,-7.158133,60.687638,90.186354,26.294940,6.025904,31.037834,0.913276,7.422948,-4.102589,...,1398.248286,895.838214,758.643714,418.044091,417.960038,285.750324,581.106789,228.024993,185.434181,288.793749
8,jazz,-11.742284,103.352755,189.690094,32.502427,-23.598896,16.748012,1.419943,-8.652519,2.056086,...,2224.929148,1792.759598,779.627176,982.699144,520.308286,428.318627,443.804904,265.378700,312.082034,272.357269
9,latin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [163]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

def compute_similarity(year, data):    
    data_year = data[year].replace(0, np.nan)

    data_year_norm = data_year.copy()
    sel_cols = data_year.columns[1:]
    for col in sel_cols:
        feature = data_year.loc[:,col]
        data_year_norm.loc[:,col] = (feature - np.mean(feature))/ np.std(feature)

    data_year_norm.fillna(0)
    data_year_norm = data_year_norm.set_index('Genre')
    data_year_norm = data_year_norm.T
    #df = pd.DataFrame(data_year_norm.corr().stack())
    df = data_year_norm.corr()
    df = normalize(df)
    #df.index = df.index.rename('Genre1')
    #df = df.stack().reset_index()
    #df.columns = ['Genre1', 'Genre2', 'corr']
    #df = df[df['Genre1'] != df['Genre2']]

    #combinations_norm = pd.DataFrame()
    #for genre in data_year['Genre'].values: 
        #combinations_norm = combinations_norm.append(normalize(df[df['Genre1'] == genre]))

    return df

In [165]:
similarity_dict = {}

for key in data2.keys():
    similarity_dict[key] = compute_similarity(key, data2)
    

In [172]:
output_file = open('similarity_dict_byDecade.dat','wb')
pickle.dump(similarity_dict, output_file)
output_file.close()

In [154]:
data_year = data2['1950s']
data_year_norm = data_year.copy()
sel_cols = data_year.columns[1:]
for col in sel_cols:
    feature = data_year.loc[:,col]
    data_year_norm.loc[:,col] = (feature - np.mean(feature))/ np.std(feature)

In [190]:
for key in similarity_dict.keys():
    similarity_dict[key].to_csv("\\Million-Song-Dataset-Visualization\\Similarity matrices flat files\\similarity_matrix_" + key + '.csv')
    